In [16]:
from typing_extensions import Annotated, TypedDict  

In [17]:
from langchain_openai import ChatOpenAI  

In [18]:
from typing import List

In [19]:
from tqdm import tqdm

In [20]:
import asyncio
from tqdm.asyncio import tqdm_asyncio

In [1]:
import pandas as pd

from langchain_core.vectorstores import InMemoryVectorStore  

from langchain_openai import OpenAIEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter  

from openai import OpenAI 

import requests

import json

from dotenv import load_dotenv
import os

# Подключаем все переменные из окружения
load_dotenv()
# Подключаем ключ для LLM-модели
LLM_API_KEY = os.getenv("LLM_API_KEY")
# Подключаем ключ для EMBEDDER-модели
EMBEDDER_API_KEY = os.getenv("EMBEDDER_API_KEY")

BASE_URL = 'https://openrouter.ai/api/v1'

client = OpenAI(
    base_url=BASE_URL,
    api_key=LLM_API_KEY,
)

questions = pd.read_csv('questions.csv')

corpus = pd.read_csv('train_data.csv')

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=BASE_URL, api_key=LLM_API_KEY)

In [5]:
from pathlib import Path

In [121]:
chunks = []
folder = Path('./my_chunks_grok//')
for fn in os.listdir(folder):
    with open(folder / fn, 'r', encoding='utf-8') as f:
        chunks.append(f.read())

In [122]:
vectorstore = await InMemoryVectorStore.afrom_texts(    
    chunks,  
    embedding=embeddings, 
)

In [123]:
vectorstore.dump('context_chunking_grok.db')

In [124]:
class Query(TypedDict): 
    question: str
    problem: str
    answer_structure: str

In [125]:
class MyKAG:
    def __init__(self, model, store): 
        self.model = model
        self.store = store
        self.queries = []
        self.results = []

    def formulate_query(self, question):
        prompt = f"""
        Из заданного тебе вопроса сформулируй структурированный запрос на векторную базу данных.
        Отвечай кратко.
        question: вопрос без изменений
        problem: проблема, которая поставлена в вопросе
        answer_structure: структура ответа, которая полностью покроет заданный вопрос. Укажи темы, которые обязательно должны быть покрыты.
        Вопрос, на который нужно ответить:
        <question>
        {question}
        </question>"""
        query = self.model.with_structured_output(Query).invoke(prompt)
        self.queries.append({
            'question': question,
            'query': query
        })
        return query
    
    def get_knowledge(self, query):
        docs = self.store.similarity_search(query=str(query), k=20)
        knowledge = '\n'.join([doc.page_content for doc in docs])
        return knowledge

    def answer(self, question):
        query = self.formulate_query(question)
        knowledge = self.get_knowledge(query)
        prompt = f"""
        Ответьте на вопрос клиента, используя только релевантную предоставленную информацию. Не используйте не относящиеся к вопросу данные. Приведите ответ на русском языке.
        Если в вопросе есть запрос на какие-то численные данные, и эти данные есть в предоставленных вам знаниях, обязательно предоставьте их. Отвечайте на вопрос так, чтобы помочь клиенту.
        Ответ на вопрос ДОЛЖЕН БЫТЬ СВЯЗАН С проблемой поставленной в вопросе:
        <Problem>
        {query['problem']}
        </Problem>
        Используйте структуру ответа:
        <Structure>
        {query['answer_structure']}
        </Structure>
        <Question>
        {question}
        </Question>
        <Knowledge>
        {knowledge}
        </Knowledge>"""
        result = self.model.invoke(prompt).content
        self.results.append({
            'question': question,
            'query': query,
            'result': result,
            'knowledge': knowledge,
            'prompt': prompt
        })
        return result

In [126]:
model = ChatOpenAI(base_url=BASE_URL, model="x-ai/grok-3-mini", api_key=LLM_API_KEY)  

In [127]:
kag = MyKAG(model, vectorstore)

In [128]:
q = questions['Вопрос'].iloc[2]
q

'Как действовать при оспаривании незаконной продажи долга с учетом установленных сроков ответа на претензии?'

In [129]:
result = kag.answer(q) 

In [130]:
print(kag.queries[0])

{'question': 'Как действовать при оспаривании незаконной продажи долга с учетом установленных сроков ответа на претензии?', 'query': {'question': 'Как действовать при оспаривании незаконной продажи долга с учетом установленных сроков ответа на претензии?', 'problem': 'Оспаривание незаконной продажи долга, с учетом соблюдения сроков для ответов на претензии, включая риски пропуска этих сроков.', 'answer_structure': 'Структура ответа должна включать: 1. Определение незаконной продажи долга и ее признаков. 2. Шаги по оспариванию (например, подача претензии, обращение в суд). 3. Установленные сроки для ответов на претензии и их последствия. 4. Необходимые документы и доказательства. 5. Возможные исходы и рекомендации.'}}


In [131]:
print(result)

### Оспаривание незаконной продажи долга: шаги и рекомендации

Я помогу вам разобраться, как действовать при оспаривании незаконной продажи долга, с учетом установленных сроков для ответов на претензии. Это важно, чтобы избежать рисков, таких как пропуск сроков, что может усложнить вашу позицию в споре. Давайте разберем по шагам, опираясь на релевантную информацию.

1. **Определение незаконной продажи долга и ее признаков**  
   Незаконная продажа долга происходит, если кредитор (например, банк или МФО) нарушил условия договора. Согласно правилам, кредиторы могут продавать долги только при просрочке выплаты, наличии согласия в исходном договоре и, для продажи физическому лицу, отдельного вашего разрешения. Признаки незаконности: долг продан без вашего ведома, без просрочки или без необходимого согласия. Это может быть ошибкой, мошенничеством или нарушением, поэтому первым делом проверьте детали долга у прежнего кредитора.

2. **Шаги по оспариванию**  
   Чтобы оспорить продажу, начните

In [132]:
print(kag.results[0]['query'])

{'question': 'Как действовать при оспаривании незаконной продажи долга с учетом установленных сроков ответа на претензии?', 'problem': 'Оспаривание незаконной продажи долга, с учетом соблюдения сроков для ответов на претензии, включая риски пропуска этих сроков.', 'answer_structure': 'Структура ответа должна включать: 1. Определение незаконной продажи долга и ее признаков. 2. Шаги по оспариванию (например, подача претензии, обращение в суд). 3. Установленные сроки для ответов на претензии и их последствия. 4. Необходимые документы и доказательства. 5. Возможные исходы и рекомендации.'}


In [133]:
print(kag.results[0]['knowledge'])

<title>
Как быстро продадут жилье на торгах?
</title>
<context>
Процедура продажи заложенного имущества включает несколько этапов через суд и приставов.
</context>
<problem>
Заемщик может потерять жилье из-за длительного процесса.
</problem>
<solution>
Процедура занимает от нескольких месяцев до года: от рассмотрения в суде до торгов; на любом этапе можно погасить долг, чтобы сохранить жилье.
</solution>
<title>
Что делать, если долг продан без вашего ведома
</title>
<context>
Если вы узнали о продаже долга от нового кредитора, это может быть ошибкой, мошенничеством или нарушением. Проверьте детали долга и свяжитесь с прежним кредитором.
</context>
<problem>
Как такое могло случиться и что теперь делать?
</problem>
<solution>
Проверьте информацию, свяжитесь с прежним кредитором. Если нарушение, откажитесь от общения с новым и подайте претензию.
</solution>
<title>
Как быстро долг оказывается у приставов?
</title>
<context>
Объясняется процесс передачи долга к приставам, который может п